In [23]:
import numpy as np
from scipy import signal
from scipy import linalg
import control
import matplotlib.pyplot as plt
import pickle

In [24]:
OSR = 256      # oversample ratio
fb = 22050     # nyquist
fs = OSR*2*fb  # sampling frequency
ts = 1/fs      # sampling period

In [25]:
file = open('filter_ss.pickle', 'rb')
[A,B,C,D] = pickle.load(file)
file.close()

## State Space Model
The shape of the state space equations determine the order of the $\Sigma\Delta$ filter. Below is the continuous time representation

$\dot{x} = A_{c}x + B_{c}u$<br>
$y = C_{c}x + D_{c}u$

The corresponding based $\delta$ model[^1].

$\delta$x = $A_{\delta}x + B_{\delta}u$<br>
$y = C_{\delta}x + D_{\delta}u$

$A_{\delta} = \dfrac{\exp(A_{c} - \textit{I})}{\Delta}$ <br>
$B_{\delta} = \dfrac{1}{\Delta}\int_{0}^{\Delta}\exp(A_{c}(t - \tau))B_{c}u(\tau)\;d\tau$ <br>
$C_{\delta} = C_{c}$ <br>
$D_{\delta} = D_{c}$ <br>

In [26]:
[A, T] = linalg.matrix_balance(A)
B = linalg.solve(T, B) 
C = C @ T

### Converting from Continuous Time to Sampled Time
c2delta - def c2delta(A,B,C,D,ts):

In [27]:
Ad = (linalg.expm(A*ts) - np.eye(A.shape[0])) / ts
Bd = np.matmul((linalg.expm(A*ts) - np.eye(A.shape[0]) ), B) / ts
Bd = np.matmul(np.linalg.inv(A), Bd)
Cd = C
Dd = D

### Structural Transformation of Filter
obsv_cst - def obsv_cst(A,B,C,D):

In [28]:
e = np.zeros((A.shape[1], 1))
e[-1] = 1
O = control.obsv(A, C)
[U, S, Vh] = linalg.svd(O)
V = Vh.T
S = np.diag(S)
S_inv = linalg.solve(S, np.eye(S.shape[0]))
T_inv = V @ S_inv @ U.conj().T
T1 = T_inv @ e;
n = A.shape[1]
q = T1.shape[1]
T0 = np.zeros((n,n))

for i in range(1, n+1):
  column = np.power(A, n-i) @ T1
  T0[:, i-1] = column[:,0]

Ad_t = linalg.solve(T0, A @ T0)
Bd_t = linalg.solve(T0, B)
Cd_t = C @ T0;
Dd_t = D;

In [29]:
[num_t, den_t] = signal.ss2tf(Ad_t,Bd_t,Cd_t,Dd_t)

In [30]:
def delta_bode(A,B,C,D,f,ts):
  q     = C.shape[0]
  p     = B.shape[1]
  fs    = 1/ts
  mag   = np.zeros((q,p,f.shape[0]))
  phz   = np.zeros((q,p,f.shape[0]))
  delta = (np.exp(1j*2*np.pi*(f/fs))-1)/ts

  for i in range(f.shape[0]):
      A_d = delta[i] * np.eye(A.shape[0]) - A
      h   = C @ linalg.solve(A_d, B) + D
      mag[:,:,i] = np.abs(h)
      phz[:,:,i] = 180*np.arctan2(np.imag(h),np.real(h))/np.pi
  return mag, phz


In [31]:
f = np.logspace(0,np.log10(fb),2**10)

### Dynamic Range Scaling of State Variable Integrators
dIIR_scaling(Ad,Bd,T0,f,ts):

In [32]:
T0_inv = linalg.solve(T0, np.eye(T0.shape[0]))
[f_int, phz] = delta_bode(A,B,T0_inv,0,f,ts)

f_norm = np.zeros(A.shape[0])

for i in range(f_norm.shape[0]):
  f_norm[i] = linalg.norm(f_int[i], np.inf, axis=1)

Ts = np.zeros(A.shape)
k = np.zeros(f_norm.shape[0])
k_inv = np.zeros(f_norm.shape[0])

for i in range(f_norm.shape[0]):
  if i == 0:
    k[i] = 1/f_norm[i]
  else:
    k[i] = 1/(np.prod(k[:i])*f_norm[i])

  k_inv[i] = 2**np.floor(np.log2(ts/k[i]))/ts
  Ts[i,i] = np.prod(k_inv[0:i+1])


In [33]:
num_ts      = np.copy(num_t[0])
num_ts[1:] /= np.diag(Ts)
num_ts[0]   = num_t[0][0]
den_ts      = np.copy(den_t)
den_ts[1:] /= np.diag(Ts)
den_ts[0]   = 1

beta  = num_ts;
alpha = den_ts;


In [34]:
beta

array([ 1.00000000e-03, -2.03551449e-12,  1.66198617e-04,  9.21224222e-07,
        2.10225865e+01])

In [35]:
alpha

array([1.00000000e+00, 1.51983976e-03, 1.84176487e-02, 1.77806186e-03,
       2.23972437e+04])

In [36]:
file = open('delta_filter_ss.pickle', 'wb')
pickle.dump((Ad,Bd,Cd,Dd,T0,Ts,f,ts), file)
file.close()

[^1]: [$\Sigma\Delta$ Stream Computation: A New Paradigm for Low Power and High Resolution Feedback Control](https://escholarship.org/uc/item/4f46n0h6) by Poverelli, Joseph Sam <br>
[^2]: [Digital Control and Estimation: A Unified Approach](https://dl.acm.org/doi/10.5555/574885) by Richard H Middleton & 
Graham C Goodwin <br>
[^3]: [A generalized direct-form delta operator-based IIR filter with minimum noise gain and sensitivity](https://ieeexplore.ieee.org/document/933811) by Ngai Wong & Tung-Sang Ng <br>